In [38]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image

class EyeDataset(Dataset):
    def __init__(self, image_dir, coordinates_dir, transform=None):
        self.image_dir = image_dir
        self.coordinates_dir = coordinates_dir
        self.transform = transform
        self.image_paths = sorted(os.listdir(image_dir))
        self.coordinate_paths = sorted(os.listdir(coordinates_dir))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.image_paths[idx])
        image = Image.open(img_name)

        coordinate_name = os.path.join(self.coordinates_dir, self.coordinate_paths[idx])
        with open(coordinate_name, 'r') as f:
            coordinates = [float(coord) for coord in f.read().strip().split()]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(coordinates)

# Usage example
from torchvision import transforms

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image
    transforms.ToTensor(),           # Convert to tensor
])

# Create dataset
dataset = EyeDataset(image_dir='frames_folder', coordinates_dir='coordinates', transform=transform)

# Example of accessing data
image, coordinates = dataset[1]
print("Image shape:", image.shape)
print("Coordinates:", coordinates)

Image shape: torch.Size([3, 224, 224])
Coordinates: tensor([ 34., 514.])


In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(256 * 14 * 14, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 2)  # Output layer for coordinates
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, 256 * 14 * 14)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Usage example
cnn_model = CNN()
print(cnn_model)


CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=50176, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=2, bias=True)
)


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm

# Define the device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [38]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [39]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 28 * 28, 64)
        self.fc2 = nn.Linear(64, 2)  # Output layer for coordinates

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, 32 * 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Instantiate the model
model = CNN().to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [40]:
num_epochs = 100
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    train_loss = 0.0
    for images, targets in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        images, targets = images.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
    
    # Validation loop
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    with torch.no_grad():
        for images, targets in val_loader:
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * images.size(0)
    
    # Print training and validation loss
    train_loss /= len(train_loader.dataset)
    val_loss /= len(val_loader.dataset)
    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

print("Training finished!")
torch.save(model.state_dict(), "small_CNN.pth")

Epoch 1/100:   0%|          | 0/415 [00:00<?, ?it/s]

Epoch 1/100: 100%|██████████| 415/415 [00:06<00:00, 60.29it/s]


Epoch 1/100, Train Loss: 55128.6999, Val Loss: 22327.9519


Epoch 2/100: 100%|██████████| 415/415 [00:07<00:00, 58.71it/s]


Epoch 2/100, Train Loss: 12648.7726, Val Loss: 10597.9069


...

Epoch 98/100, Train Loss: 251.8296, Val Loss: 1592.6256


Epoch 99/100: 100%|██████████| 415/415 [00:05<00:00, 72.31it/s]


Epoch 99/100, Train Loss: 212.3754, Val Loss: 1735.0923


Epoch 100/100: 100%|██████████| 415/415 [00:06<00:00, 68.66it/s]


Epoch 100/100, Train Loss: 183.3518, Val Loss: 1677.3004
Training finished!


In [45]:
image_path = "frames_folder/300.jpg"
image = Image.open(image_path)

# Preprocess the image
input_image = transform(image).unsqueeze(0)
with torch.no_grad():
    model.eval()  # Set the model to evaluation mode
    output = model(input_image.to(device))

# Print the output
print("Model Output:")
print(output)

Model Output:
tensor([[ 0.0206, -0.0130]], device='cuda:0')


In [46]:
# Define the file path where the model is saved
model_path = "small_CNN.pth"

# Reconstruct the model instance
model = CNN()  # Assuming CNN is your model class
model = CNN().to(device)
# Load the saved model state dictionary
model.load_state_dict(torch.load(model_path))

# Ensure the model is in evaluation mode
model.eval()

print("Model loaded successfully.")


Model loaded successfully.


In [48]:
from PIL import Image
import torch
import torchvision.transforms as transforms

# Assuming you have already loaded the model and it's in the variable 'model'

# Load the image
image_path = "frames_folder/410.jpg"
image = Image.open(image_path)

# Define the transformation to apply to the image
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image
    transforms.ToTensor(),           # Convert to tensor
])
# Preprocess the image
input_image = transform(image).unsqueeze(0)
input_image = input_image.to(device)
# Perform inference
with torch.no_grad():
    model.eval()  # Set the model to evaluation mode
    output = model(input_image)

# Process the output if needed
# For example, convert output tensor to numpy array
output_np = output.cpu().numpy()

# Print the output
print("Model Output:")
print(output)


Model Output:
tensor([[384.1263, 692.3995]], device='cuda:0')


In [ ]:
print(val_dataset[0][1])

tensor([366., 566.])


In [ ]:
val_dataset_img = val_dataset[0][0]

# Assuming val_dataset_img is a torch tensor, convert it to a numpy array
val_dataset_img_np = val_dataset_img.numpy()
input_image = transform(val_dataset_img_np).unsqueeze(0)
input_image = input_image.to(device)
with torch.no_grad():
    model.eval()  # Set the model to evaluation mode
    output = model(input_image)
print(output)

TypeError: Unexpected type <class 'numpy.ndarray'>

In [3]:
import os
import shutil

# Define the paths to the folders
coordinates_folder = "coordinates"
frames_folder = "frames_folder"
frames2_folder = "frames2_folder"

# Create three new folders to store the selected files
val_coordinates_folder = "val_coordinates"
val_frames_folder = "val_frames"
val_frames2_folder = "val_frames2"

# Create three new folders to store the unselected files
train_coordinates_folder = "train_coordinates"
train_frames_folder = "train_frames"
train_frames2_folder = "train_frames2"

# Create the output and unselected folders if they don't exist
for folder in [val_coordinates_folder, val_frames_folder, val_frames2_folder,
               train_coordinates_folder, train_frames_folder, train_frames2_folder]:
    os.makedirs(folder, exist_ok=True)

# Define the number of files to select
num_files = 519

# Iterate over the range of files (0 to num_files)
for i in range(num_files):
    # Define the file names to copy
    coordinates_file = f"coordinates_{i}.txt"
    frame_file = f"{i}.jpg"
    
    # Construct the paths to the files
    coordinates_file_path = os.path.join(coordinates_folder, coordinates_file)
    frame_file_path = os.path.join(frames_folder, frame_file)
    frame2_file_path = os.path.join(frames2_folder, frame_file)
    
    # Check if all three files exist
    if os.path.exists(coordinates_file_path) and os.path.exists(frame_file_path) and os.path.exists(frame2_file_path):
        # Check if it's every 5th image
        if i % 5 == 0:
            # Copy the files to the validation folders
            shutil.copy(coordinates_file_path, val_coordinates_folder)
            shutil.copy(frame_file_path, val_frames_folder)
            shutil.copy(frame2_file_path, val_frames2_folder)
            print(f"Copied files {coordinates_file}, {frame_file}, and {frame_file} to respective validation folders.")
        else:
            # Copy the files to the training folders
            shutil.copy(coordinates_file_path, train_coordinates_folder)
            shutil.copy(frame_file_path, train_frames_folder)
            shutil.copy(frame2_file_path, train_frames2_folder)
            print(f"Copied files {coordinates_file}, {frame_file}, and {frame_file} to respective training folders.")
    else:
        # If any of the files is missing, copy them to the unselected folders
        if not os.path.exists(coordinates_file_path):
            shutil.copy(coordinates_file_path, train_coordinates_folder)
        if not os.path.exists(frame_file_path):
            shutil.copy(frame_file_path, train_frames_folder)
        if not os.path.exists(frame2_file_path):
            shutil.copy(frame2_file_path, train_frames2_folder)
        print(f"Files {coordinates_file}, {frame_file}, and {frame_file} are missing and copied to respective training folders.")

print("Files copied successfully.")


Copied files coordinates_0.txt, 0.jpg, and 0.jpg to respective validation folders.
Copied files coordinates_1.txt, 1.jpg, and 1.jpg to respective training folders.
Copied files coordinates_2.txt, 2.jpg, and 2.jpg to respective training folders.
Copied files coordinates_3.txt, 3.jpg, and 3.jpg to respective training folders.
Copied files coordinates_4.txt, 4.jpg, and 4.jpg to respective training folders.
Copied files coordinates_5.txt, 5.jpg, and 5.jpg to respective validation folders.
Copied files coordinates_6.txt, 6.jpg, and 6.jpg to respective training folders.
Copied files coordinates_7.txt, 7.jpg, and 7.jpg to respective training folders.
Copied files coordinates_8.txt, 8.jpg, and 8.jpg to respective training folders.
Copied files coordinates_9.txt, 9.jpg, and 9.jpg to respective training folders.
Copied files coordinates_10.txt, 10.jpg, and 10.jpg to respective validation folders.
Copied files coordinates_11.txt, 11.jpg, and 11.jpg to respective training folders.
Copied files coo

CNN_1 Training


In [1]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image

class EyeDataset(Dataset):
    def __init__(self, image_dir, coordinates_dir, transform=None):
        self.image_dir = image_dir
        self.coordinates_dir = coordinates_dir
        self.transform = transform
        self.image_paths = sorted(os.listdir(image_dir))
        self.coordinate_paths = sorted(os.listdir(coordinates_dir))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.image_paths[idx])
        image = Image.open(img_name)

        coordinate_name = os.path.join(self.coordinates_dir, self.coordinate_paths[idx])
        with open(coordinate_name, 'r') as f:
            coordinates = [float(coord) for coord in f.read().strip().split()]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(coordinates)

# Usage example
from torchvision import transforms

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image
    transforms.ToTensor(),           # Convert to tensor
    transforms.Normalize(mean=[0.389 , 0.389, 0.389], std=[0.207, 0.207, 0.207]),  # Normalize
])


# Create dataset
dataset = EyeDataset(image_dir='train_frames', coordinates_dir='train_coordinates', transform=transform)

# Example of accessing data
image, coordinates = dataset[1]
print("Image shape:", image.shape)
print("Coordinates:", coordinates)

Image shape: torch.Size([3, 224, 224])
Coordinates: tensor([553., 502.])


In [3]:
import torch
import numpy as np

# Load dataset without transformations
dataset_no_transform = EyeDataset(image_dir='train_frames', coordinates_dir='train_coordinates')

# Initialize lists to store pixel values
pixel_values = []

# Iterate over the dataset to collect pixel values
for i in range(len(dataset_no_transform)):
    image, _ = dataset_no_transform[i]  # Ignore coordinates
    image_tensor = transforms.ToTensor()(image)  # Convert PIL Image to PyTorch tensor
    pixel_values.append(image_tensor.view(-1, 3))  # Flatten image tensor and append to list

# Concatenate pixel values from all images
pixel_values = torch.cat(pixel_values, dim=0)

# Compute mean and standard deviation
mean = torch.mean(pixel_values, dim=0)
std = torch.std(pixel_values, dim=0)

# Convert to numpy arrays
mean = mean.numpy()
std = std.numpy()

print("Mean:", mean)
print("Standard Deviation:", std)


TypeError: pic should be PIL Image or ndarray. Got <class 'torch.Tensor'>

In [48]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)  # Update input size for fc1
        self.fc2 = nn.Linear(128, 2)  # Output layer for coordinates
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, 64 * 7 * 7)  # Update view size
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CNN().to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm

# Define the device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [50]:
train_size = int(1 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [51]:
num_epochs = 200
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    train_loss = 0.0
    for images, targets in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        images, targets = images.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
    
    # Validation loop
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        for images, targets in val_loader:
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * images.size(0)
    
    # Print training and validation loss
    train_loss /= len(train_loader.dataset)
    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}")

print("Training finished!")
torch.save(model.state_dict(), "models/CNN_1.pth")

Epoch 1/200:   0%|          | 0/415 [00:00<?, ?it/s]c:\Users\wwwth\anaconda3\envs\ultra\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1, 2])) that is different to the input size (torch.Size([16, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch 1/200: 100%|██████████| 415/415 [00:07<00:00, 53.49it/s]


Epoch 1/200, Train Loss: 59928.3045


Epoch 2/200: 100%|██████████| 415/415 [00:05<00:00, 69.47it/s]


Epoch 2/200, Train Loss: 14535.3069


Epoch 3/200: 100%|██████████| 415/415 [00:06<00:00, 64.96it/s]


...

Epoch 198/200, Train Loss: 461.4481


Epoch 199/200: 100%|██████████| 415/415 [00:05<00:00, 72.71it/s]


Epoch 199/200, Train Loss: 563.6492


Epoch 200/200: 100%|██████████| 415/415 [00:05<00:00, 72.53it/s]

Epoch 200/200, Train Loss: 386.1693
Training finished!


accuracy


In [52]:
image_path = "val_frames/300.jpg"
image = Image.open(image_path)

# Preprocess the image
input_image = transform(image).unsqueeze(0)
with torch.no_grad():
    model.eval()  # Set the model to evaluation mode
    output = model(input_image.to(device))

# Print the output
print("Model Output:")
print(output)

Model Output:
tensor([[490.1335, 556.4156],
        [487.8913, 553.7632],
        [490.4862, 559.4265],
        [493.0864, 558.4758],
        [489.7811, 555.3039],
        [491.8326, 557.2448],
        [489.5594, 556.0657],
        [490.0649, 556.1379],
        [489.9196, 557.7335],
        [489.8178, 556.1094],
        [496.4900, 563.7261],
        [492.6862, 558.4260],
        [489.6976, 555.9413],
        [489.5376, 555.7931],
        [488.8633, 555.0364],
        [493.7497, 557.1525]], device='cuda:0')


In [53]:
import os
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import numpy as np
class MultiEyeDataset(Dataset):
    def __init__(self, image_dirs, coordinates_dirs, transform=None):
        self.image_dirs = image_dirs
        self.coordinates_dirs = coordinates_dirs
        self.transform = transform
        self.image_paths = []
        self.coordinate_paths = []
        
        for img_dir, coord_dir in zip(image_dirs, coordinates_dirs):
            self.image_paths += sorted(os.listdir(img_dir))
            self.coordinate_paths += sorted(os.listdir(coord_dir))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dirs[0], self.image_paths[idx])
        image = Image.open(img_name)

        coordinate_name = os.path.join(self.coordinates_dirs[0], self.coordinate_paths[idx])
        with open(coordinate_name, 'r') as f:
            coordinates = [float(coord) for coord in f.read().strip().split()]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(coordinates)

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image
    transforms.ToTensor(),           # Convert to tensor
])

# Create dataset for additional validation folders
val_dataset2 = MultiEyeDataset(
    image_dirs=['val_frames'],
    coordinates_dirs=['val_coordinates'],
    transform=transform
)
# Create data loader for the additional validation dataset
val_loader2 = DataLoader(val_dataset2, batch_size=1, shuffle=False)

# Load the trained model
model = CNN().to(device)
model.load_state_dict(torch.load("models/CNN_1.pth"))
model.eval()

# Calculate accuracy on the additional validation dataset



CNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=3136, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

In [60]:
def calculate_accuracy(predicted, target):
    distance = abs(predicted - target)

    #accuracy = 100.0 - (distance / threshold * 100.0)
    #print(distance)
    return distance # Ensure accuracy is not negative

# Set distance threshold for accuracy calculation
numbers = 0
total = 0
with torch.no_grad():
    for images, targets in val_loader2:
        images, targets = images.to(device), targets.to(device)
        outputs = model(images)
        # Assuming output is in the format [x, y]
        predicted_coordinates = outputs.cpu().numpy()[0]
        true_coordinates = targets.cpu().numpy()[0]
        # Calculate accuracy
        normal = calculate_accuracy(predicted_coordinates, true_coordinates)
        total += normal
        numbers += 1
        #if accuracy >= 0:  # Exclude cases where accuracy is negative
        #    correct += 1
        #total += 1

accuracy_percentagex = (total[0]/numbers) 
accuracy_percentagey = (total[1]/numbers) 
print(f"Percentage accuracy x: {accuracy_percentagex:.2f}")
print(f"Percentage accuracy x: {accuracy_percentagey:.2f}")

Percentage accuracy x: 27.81
Percentage accuracy x: 29.20


CNN 2

In [11]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(256 * 14 * 14, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 2)  # Output layer for coordinates
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, 256 * 14 * 14)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# Instantiate the model
model = CNN().to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [12]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image

class EyeDataset(Dataset):
    def __init__(self, image_dir, coordinates_dir, transform=None):
        self.image_dir = image_dir
        self.coordinates_dir = coordinates_dir
        self.transform = transform
        self.image_paths = sorted(os.listdir(image_dir))
        self.coordinate_paths = sorted(os.listdir(coordinates_dir))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.image_paths[idx])
        image = Image.open(img_name)

        coordinate_name = os.path.join(self.coordinates_dir, self.coordinate_paths[idx])
        with open(coordinate_name, 'r') as f:
            coordinates = [float(coord) for coord in f.read().strip().split()]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(coordinates)

# Usage example
from torchvision import transforms

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image
    transforms.ToTensor(),           # Convert to tensor
    transforms.Normalize(mean=[0.389 , 0.389, 0.389], std=[0.207, 0.207, 0.207]),  # Normalize
])

# Create dataset
dataset = EyeDataset(image_dir='train_frames', coordinates_dir='train_coordinates', transform=transform)

# Example of accessing data
image, coordinates = dataset[1]
print("Image shape:", image.shape)
print("Coordinates:", coordinates)

Image shape: torch.Size([3, 224, 224])
Coordinates: tensor([553., 502.])


In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm

# Define the device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [14]:
train_size = int(1 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
num_epochs = 200
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    train_loss = 0.0
    for images, targets in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        images, targets = images.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
    
    # Validation loop
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        for images, targets in val_loader:
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * images.size(0)
    
    # Print training and validation loss
    train_loss /= len(train_loader.dataset)
    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}")

print("Training finished!")
torch.save(model.state_dict(), "models/CNN_2_normalized_200epochs.pth")

Epoch 1/200: 100%|██████████| 847/847 [00:29<00:00, 28.92it/s]


Epoch 1/200, Train Loss: 29643.0841


Epoch 2/200: 100%|██████████| 847/847 [00:25<00:00, 32.83it/s]


Epoch 2/200, Train Loss: 10079.9658


Epoch 3/200: 100%|██████████| 847/847 [00:24<00:00, 35.13it/s]


...

Epoch 198/200, Train Loss: 107.3720


Epoch 199/200: 100%|██████████| 847/847 [00:21<00:00, 39.02it/s]


Epoch 199/200, Train Loss: 94.7865


Epoch 200/200: 100%|██████████| 847/847 [00:21<00:00, 39.13it/s]


Epoch 200/200, Train Loss: 125.0694
Training finished!


In [20]:
import torch.nn.functional as F
image_path = "val_frames/300.jpg"
image = Image.open(image_path)

# Preprocess the image
input_image = transform(image).unsqueeze(0)
with torch.no_grad():
    model.eval()  # Set the model to evaluation mode
    output = model(input_image.to(device))

# Print the output
print("Model Output:")
print(output)

Model Output:
tensor([[495.6587, 501.0004]], device='cuda:0')


In [24]:
import os
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import numpy as np
class MultiEyeDataset(Dataset):
    def __init__(self, image_dirs, coordinates_dirs, transform=None):
        self.image_dirs = image_dirs
        self.coordinates_dirs = coordinates_dirs
        self.transform = transform
        self.image_paths = []
        self.coordinate_paths = []
        
        for img_dir, coord_dir in zip(image_dirs, coordinates_dirs):
            self.image_paths += sorted(os.listdir(img_dir))
            self.coordinate_paths += sorted(os.listdir(coord_dir))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dirs[0], self.image_paths[idx])
        image = Image.open(img_name)

        coordinate_name = os.path.join(self.coordinates_dirs[0], self.coordinate_paths[idx])
        with open(coordinate_name, 'r') as f:
            coordinates = [float(coord) for coord in f.read().strip().split()]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(coordinates)

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image
    transforms.ToTensor(),           # Convert to tensor
    transforms.Normalize(mean=[0.389 , 0.389, 0.389], std=[0.207, 0.207, 0.207]),  # Normalize
])

# Create dataset for additional validation folders
val_dataset2 = MultiEyeDataset(
    image_dirs=['val_frames'],
    coordinates_dirs=['val_coordinates'],
    transform=transform
)
# Create data loader for the additional validation dataset
val_loader2 = DataLoader(val_dataset2, batch_size=1, shuffle=False)

# Load the trained model
model = CNN().to(device)
model.load_state_dict(torch.load("models/CNN_2_normalized_200epochs.pth"))
model.eval()

# Calculate accuracy on the additional validation dataset

CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=50176, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=2, bias=True)
)

In [25]:
def calculate_accuracy(predicted, target):
    distance = abs(predicted - target)

    #accuracy = 100.0 - (distance / threshold * 100.0)
    #print(distance)
    return distance # Ensure accuracy is not negative

# Set distance threshold for accuracy calculation
numbers = 0
total = 0
with torch.no_grad():
    for images, targets in val_loader2:
        images, targets = images.to(device), targets.to(device)
        
        outputs = model(images)
        # Assuming output is in the format [x, y]
        predicted_coordinates = outputs.cpu().numpy()[0]
        true_coordinates = targets.cpu().numpy()[0]
        # Calculate accuracy
        normal = calculate_accuracy(predicted_coordinates, true_coordinates)
        total += normal
        numbers += 1
        #if accuracy >= 0:  # Exclude cases where accuracy is negative
        #    correct += 1
        #total += 1

accuracy_percentagex = (total[0]/numbers) 
accuracy_percentagey = (total[1]/numbers) 
print(f"Percentage accuracy x: {accuracy_percentagex:.2f}")
print(f"Percentage accuracy x: {accuracy_percentagey:.2f}")

Percentage accuracy x: 30.69
Percentage accuracy x: 23.72


CNN 3


In [75]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 28 * 28, 64)
        self.fc2 = nn.Linear(64, 2)  # Output layer for coordinates

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, 32 * 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    # Instantiate the model
model = CNN().to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [76]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image

class EyeDataset(Dataset):
    def __init__(self, image_dir, coordinates_dir, transform=None):
        self.image_dir = image_dir
        self.coordinates_dir = coordinates_dir
        self.transform = transform
        self.image_paths = sorted(os.listdir(image_dir))
        self.coordinate_paths = sorted(os.listdir(coordinates_dir))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.image_paths[idx])
        image = Image.open(img_name)

        coordinate_name = os.path.join(self.coordinates_dir, self.coordinate_paths[idx])
        with open(coordinate_name, 'r') as f:
            coordinates = [float(coord) for coord in f.read().strip().split()]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(coordinates)

# Usage example
from torchvision import transforms

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image
    transforms.ToTensor(),           # Convert to tensor
])

# Create dataset
dataset = EyeDataset(image_dir='train_frames', coordinates_dir='train_coordinates', transform=transform)

# Example of accessing data
image, coordinates = dataset[1]
print("Image shape:", image.shape)
print("Coordinates:", coordinates)

Image shape: torch.Size([3, 224, 224])
Coordinates: tensor([637., 583.])


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm

# Define the device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [73]:
train_size = int(1 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [78]:
num_epochs = 200
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    train_loss = 0.0
    for images, targets in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        images, targets = images.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
    
    # Validation loop
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        for images, targets in val_loader:
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * images.size(0)
    
    # Print training and validation loss
    train_loss /= len(train_loader.dataset)
    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}")

print("Training finished!")
torch.save(model.state_dict(), "models/CNN_3.pth")

Epoch 1/200: 100%|██████████| 415/415 [00:05<00:00, 73.22it/s]


Epoch 1/200, Train Loss: 62624.7748


Epoch 2/200: 100%|██████████| 415/415 [00:05<00:00, 82.94it/s]


Epoch 2/200, Train Loss: 17536.6564


Epoch 3/200: 100%|██████████| 415/415 [00:04<00:00, 92.39it/s]


...

Epoch 198/200, Train Loss: 106.2183


Epoch 199/200: 100%|██████████| 415/415 [00:04<00:00, 88.28it/s]


Epoch 199/200, Train Loss: 87.8500


Epoch 200/200: 100%|██████████| 415/415 [00:04<00:00, 87.03it/s]


Epoch 200/200, Train Loss: 125.4275
Training finished!


In [79]:
image_path = "val_frames/300.jpg"
image = Image.open(image_path)

# Preprocess the image
input_image = transform(image).unsqueeze(0)
with torch.no_grad():
    model.eval()  # Set the model to evaluation mode
    output = model(input_image.to(device))

# Print the output
print("Model Output:")
print(output)

Model Output:
tensor([[493.6790, 520.7700]], device='cuda:0')


In [81]:
import os
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import numpy as np
class MultiEyeDataset(Dataset):
    def __init__(self, image_dirs, coordinates_dirs, transform=None):
        self.image_dirs = image_dirs
        self.coordinates_dirs = coordinates_dirs
        self.transform = transform
        self.image_paths = []
        self.coordinate_paths = []
        
        for img_dir, coord_dir in zip(image_dirs, coordinates_dirs):
            self.image_paths += sorted(os.listdir(img_dir))
            self.coordinate_paths += sorted(os.listdir(coord_dir))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dirs[0], self.image_paths[idx])
        image = Image.open(img_name)

        coordinate_name = os.path.join(self.coordinates_dirs[0], self.coordinate_paths[idx])
        with open(coordinate_name, 'r') as f:
            coordinates = [float(coord) for coord in f.read().strip().split()]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(coordinates)

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image
    transforms.ToTensor(),           # Convert to tensor
])

# Create dataset for additional validation folders
val_dataset2 = MultiEyeDataset(
    image_dirs=['val_frames'],
    coordinates_dirs=['val_coordinates'],
    transform=transform
)
# Create data loader for the additional validation dataset
val_loader2 = DataLoader(val_dataset2, batch_size=1, shuffle=False)

# Load the trained model
model = CNN().to(device)
model.load_state_dict(torch.load("models/CNN_3.pth"))
model.eval()

# Calculate accuracy on the additional validation dataset

CNN(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=25088, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=2, bias=True)
)

In [82]:
def calculate_accuracy(predicted, target):
    distance = abs(predicted - target)

    #accuracy = 100.0 - (distance / threshold * 100.0)
    #print(distance)
    return distance # Ensure accuracy is not negative

# Set distance threshold for accuracy calculation
numbers = 0
total = 0
with torch.no_grad():
    for images, targets in val_loader2:
        images, targets = images.to(device), targets.to(device)
        outputs = model(images)
        # Assuming output is in the format [x, y]
        predicted_coordinates = outputs.cpu().numpy()[0]
        true_coordinates = targets.cpu().numpy()[0]
        # Calculate accuracy
        normal = calculate_accuracy(predicted_coordinates, true_coordinates)
        total += normal
        numbers += 1
        #if accuracy >= 0:  # Exclude cases where accuracy is negative
        #    correct += 1
        #total += 1

accuracy_percentagex = (total[0]/numbers) 
accuracy_percentagey = (total[1]/numbers) 
print(f"Percentage accuracy x: {accuracy_percentagex:.2f}")
print(f"Percentage accuracy x: {accuracy_percentagey:.2f}")

Percentage accuracy x: 25.22
Percentage accuracy x: 20.39


CNN 4

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(512 * 14 * 14, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 2)  # Output layer for coordinates
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, 512 * 14 * 14)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# Instantiate the model
model = CNN().to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [6]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image

class EyeDataset(Dataset):
    def __init__(self, image_dir, coordinates_dir, transform=None):
        self.image_dir = image_dir
        self.coordinates_dir = coordinates_dir
        self.transform = transform
        self.image_paths = sorted(os.listdir(image_dir))
        self.coordinate_paths = sorted(os.listdir(coordinates_dir))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.image_paths[idx])
        image = Image.open(img_name)

        coordinate_name = os.path.join(self.coordinates_dir, self.coordinate_paths[idx])
        with open(coordinate_name, 'r') as f:
            coordinates = [float(coord) for coord in f.read().strip().split()]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(coordinates)

# Usage example
from torchvision import transforms

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image
    transforms.ToTensor(),           # Convert to tensor
])

# Create dataset
dataset = EyeDataset(image_dir='train_frames', coordinates_dir='train_coordinates', transform=transform)

# Example of accessing data
image, coordinates = dataset[1]
print("Image shape:", image.shape)
print("Coordinates:", coordinates)

Image shape: torch.Size([3, 224, 224])
Coordinates: tensor([637., 583.])


In [7]:
train_size = int(1 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [8]:
num_epochs = 200
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    train_loss = 0.0
    for images, targets in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        images, targets = images.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
    
    # Validation loop
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        for images, targets in val_loader:
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * images.size(0)
    
    # Print training and validation loss
    train_loss /= len(train_loader.dataset)
    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}")

print("Training finished!")
torch.save(model.state_dict(), "models/CNN_4.pth")

Epoch 1/200: 100%|██████████| 415/415 [00:33<00:00, 12.52it/s]


Epoch 1/200, Train Loss: 82961.0199


Epoch 2/200: 100%|██████████| 415/415 [00:29<00:00, 13.84it/s]


Epoch 2/200, Train Loss: 14894.8830


Epoch 3/200: 100%|██████████| 415/415 [00:30<00:00, 13.43it/s]


...

Epoch 198/200, Train Loss: 143.7694


Epoch 199/200: 100%|██████████| 415/415 [00:30<00:00, 13.78it/s]


Epoch 199/200, Train Loss: 127.0787


Epoch 200/200: 100%|██████████| 415/415 [00:29<00:00, 13.86it/s]


Epoch 200/200, Train Loss: 153.3962
Training finished!


In [9]:
image_path = "val_frames/300.jpg"
image = Image.open(image_path)

# Preprocess the image
input_image = transform(image).unsqueeze(0)
with torch.no_grad():
    model.eval()  # Set the model to evaluation mode
    output = model(input_image.to(device))

# Print the output
print("Model Output:")
print(output)

Model Output:
tensor([[495.5703, 557.1188]], device='cuda:0')


In [10]:
import os
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import numpy as np
class MultiEyeDataset(Dataset):
    def __init__(self, image_dirs, coordinates_dirs, transform=None):
        self.image_dirs = image_dirs
        self.coordinates_dirs = coordinates_dirs
        self.transform = transform
        self.image_paths = []
        self.coordinate_paths = []
        
        for img_dir, coord_dir in zip(image_dirs, coordinates_dirs):
            self.image_paths += sorted(os.listdir(img_dir))
            self.coordinate_paths += sorted(os.listdir(coord_dir))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dirs[0], self.image_paths[idx])
        image = Image.open(img_name)

        coordinate_name = os.path.join(self.coordinates_dirs[0], self.coordinate_paths[idx])
        with open(coordinate_name, 'r') as f:
            coordinates = [float(coord) for coord in f.read().strip().split()]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(coordinates)

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image
    transforms.ToTensor(),           # Convert to tensor
])

# Create dataset for additional validation folders
val_dataset2 = MultiEyeDataset(
    image_dirs=['val_frames'],
    coordinates_dirs=['val_coordinates'],
    transform=transform
)
# Create data loader for the additional validation dataset
val_loader2 = DataLoader(val_dataset2, batch_size=1, shuffle=False)

# Load the trained model
model = CNN().to(device)
model.load_state_dict(torch.load("models/CNN_4.pth"))
model.eval()

# Calculate accuracy on the additional validation dataset

CNN(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=100352, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=2, bias=True)
)

In [11]:
def calculate_accuracy(predicted, target):
    distance = abs(predicted - target)

    #accuracy = 100.0 - (distance / threshold * 100.0)
    #print(distance)
    return distance # Ensure accuracy is not negative

# Set distance threshold for accuracy calculation
numbers = 0
total = 0
with torch.no_grad():
    for images, targets in val_loader2:
        images, targets = images.to(device), targets.to(device)
        outputs = model(images)
        # Assuming output is in the format [x, y]
        predicted_coordinates = outputs.cpu().numpy()[0]
        true_coordinates = targets.cpu().numpy()[0]
        # Calculate accuracy
        normal = calculate_accuracy(predicted_coordinates, true_coordinates)
        total += normal
        numbers += 1
        #if accuracy >= 0:  # Exclude cases where accuracy is negative
        #    correct += 1
        #total += 1

accuracy_percentagex = (total[0]/numbers) 
accuracy_percentagey = (total[1]/numbers) 
print(f"Percentage accuracy x: {accuracy_percentagex:.2f}")
print(f"Percentage accuracy x: {accuracy_percentagey:.2f}")

Percentage accuracy x: 24.46
Percentage accuracy x: 20.48


CNN 5


In [55]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(256 * 14 * 14, 1024)  # Increased size of the first FC layer
        self.fc2 = nn.Linear(1024, 512)  # Increased size of the second FC layer
        self.fc3 = nn.Linear(512, 128)  # Increased size of the third FC layer
        self.fc4 = nn.Linear(128, 2)  # Output layer for coordinates
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, 256 * 14 * 14)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))  # Applying ReLU activation to the third FC layer
        x = self.fc4(x)
        return x



# Instantiate the model
model = CNN().to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [56]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image

class EyeDataset(Dataset):
    def __init__(self, image_dir, coordinates_dir, transform=None):
        self.image_dir = image_dir
        self.coordinates_dir = coordinates_dir
        self.transform = transform
        self.image_paths = sorted(os.listdir(image_dir))
        self.coordinate_paths = sorted(os.listdir(coordinates_dir))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.image_paths[idx])
        image = Image.open(img_name)

        coordinate_name = os.path.join(self.coordinates_dir, self.coordinate_paths[idx])
        with open(coordinate_name, 'r') as f:
            coordinates = [float(coord) for coord in f.read().strip().split()]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(coordinates)

# Usage example
from torchvision import transforms

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image
    transforms.ToTensor(),           # Convert to tensor
])

# Create dataset
dataset = EyeDataset(image_dir='train_frames', coordinates_dir='train_coordinates', transform=transform)

# Example of accessing data
image, coordinates = dataset[1]
print("Image shape:", image.shape)
print("Coordinates:", coordinates)

Image shape: torch.Size([3, 224, 224])
Coordinates: tensor([553., 502.])


In [57]:
train_size = int(1 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [60]:
num_epochs = 50
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    train_loss = 0.0
    for images, targets in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        images, targets = images.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
    
    # Validation loop
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        for images, targets in val_loader:
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * images.size(0)
    
    # Print training and validation loss
    train_loss /= len(train_loader.dataset)
    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}")

print("Training finished!")
torch.save(model.state_dict(), "models/CNN_5.pth")

Epoch 1/50: 100%|██████████| 847/847 [00:32<00:00, 26.02it/s]


Epoch 1/50, Train Loss: 204.8048


Epoch 2/50: 100%|██████████| 847/847 [00:32<00:00, 26.40it/s]


Epoch 2/50, Train Loss: 311.1246


Epoch 3/50: 100%|██████████| 847/847 [00:33<00:00, 25.32it/s]


...

Epoch 48/50, Train Loss: 179.4278


Epoch 49/50: 100%|██████████| 847/847 [00:33<00:00, 24.93it/s]


Epoch 49/50, Train Loss: 161.6914


Epoch 50/50: 100%|██████████| 847/847 [00:33<00:00, 25.28it/s]


Epoch 50/50, Train Loss: 192.2216
Training finished!


In [61]:
image_path = "val_frames/300.jpg"
image = Image.open(image_path)

# Preprocess the image
input_image = transform(image).unsqueeze(0)
with torch.no_grad():
    model.eval()  # Set the model to evaluation mode
    output = model(input_image.to(device))

# Print the output
print("Model Output:")
print(output)

Model Output:
tensor([[494.5521, 542.6638]], device='cuda:0')


In [64]:
import os
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import numpy as np
class MultiEyeDataset(Dataset):
    def __init__(self, image_dirs, coordinates_dirs, transform=None):
        self.image_dirs = image_dirs
        self.coordinates_dirs = coordinates_dirs
        self.transform = transform
        self.image_paths = []
        self.coordinate_paths = []
        
        for img_dir, coord_dir in zip(image_dirs, coordinates_dirs):
            self.image_paths += sorted(os.listdir(img_dir))
            self.coordinate_paths += sorted(os.listdir(coord_dir))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dirs[0], self.image_paths[idx])
        image = Image.open(img_name)

        coordinate_name = os.path.join(self.coordinates_dirs[0], self.coordinate_paths[idx])
        with open(coordinate_name, 'r') as f:
            coordinates = [float(coord) for coord in f.read().strip().split()]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(coordinates)

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image
    transforms.ToTensor(),           # Convert to tensor
])

# Create dataset for additional validation folders
val_dataset2 = MultiEyeDataset(
    image_dirs=['val_frames'],
    coordinates_dirs=['val_coordinates'],
    transform=transform
)
# Create data loader for the additional validation dataset
val_loader2 = DataLoader(val_dataset2, batch_size=1, shuffle=False)

# Load the trained model
model = CNN().to(device)
model.load_state_dict(torch.load("models/CNN_5.pth"))
model.eval()

# Calculate accuracy on the additional validation dataset

CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=50176, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=2, bias=True)
)

In [65]:
def calculate_accuracy(predicted, target):
    distance = abs(predicted - target)

    #accuracy = 100.0 - (distance / threshold * 100.0)
    #print(distance)
    return distance # Ensure accuracy is not negative

# Set distance threshold for accuracy calculation
numbers = 0
total = 0
with torch.no_grad():
    for images, targets in val_loader2:
        images, targets = images.to(device), targets.to(device)
        outputs = model(images)
        # Assuming output is in the format [x, y]
        predicted_coordinates = outputs.cpu().numpy()[0]
        true_coordinates = targets.cpu().numpy()[0]
        # Calculate accuracy
        normal = calculate_accuracy(predicted_coordinates, true_coordinates)
        total += normal
        numbers += 1
        #if accuracy >= 0:  # Exclude cases where accuracy is negative
        #    correct += 1
        #total += 1

accuracy_percentagex = (total[0]/numbers) 
accuracy_percentagey = (total[1]/numbers) 
print(f"Percentage accuracy x: {accuracy_percentagex:.2f}")
print(f"Percentage accuracy x: {accuracy_percentagey:.2f}")

Percentage accuracy x: 24.15
Percentage accuracy x: 23.13
